In [1]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import cv2

from urllib import request
import urllib
from io import BytesIO
import numpy as np
from skimage import io
from tqdm import tqdm

In [2]:
path = "./csv/식탁-주방가구/식탁의자"

In [3]:
df = pd.read_csv(path+".csv")
df.head()

,PROD_ID,PROD_NM,LV0,LV1,LV2,IMG_URL,LV0_onehot,LV1_onehot,LV2_onehot
0,86041,한샘몰최저가[동서가구]2/4/대리석 식탁모음,식탁/주방가구,식탁,2인,https://image.hanssem.com/hsimg//gds/550/86/86...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,92309,[유캐슬] 카레나 엔틱 식탁세트_6인,식탁/주방가구,식탁,6인이상,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
2,92310,[유캐슬] 카레나 엔틱 식탁세트_4인,식탁/주방가구,식탁,4인,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,92311,[유캐슬] 루벤스 엔틱 식탁세트_6인,식탁/주방가구,식탁,6인이상,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
4,92312,[유캐슬] 루벤스 엔틱 식탁세트_4인,식탁/주방가구,식탁,4인,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [4]:
len(df)

3102

In [5]:

df["LV2"].unique()

array(['2인', '6인이상', '4인', '확장형/기능성식탁', '식탁', '바테이블'], dtype=object)

In [6]:
for label in df["LV2"].unique():
    if not os.path.isdir(path):
        os.makedirs(path)

In [7]:
for idx in df.index:
    df["LV0"][idx] = df["LV0"][idx].replace("/","+")
    df["LV1"][idx] = df["LV1"][idx].replace("/","+")
    df["LV2"][idx] = df["LV2"][idx].replace("/","+")

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
df.head()

,PROD_ID,PROD_NM,LV0,LV1,LV2,IMG_URL,LV0_onehot,LV1_onehot,LV2_onehot
0,86041,한샘몰최저가[동서가구]2/4/대리석 식탁모음,식탁+주방가구,식탁,2인,https://image.hanssem.com/hsimg//gds/550/86/86...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...
1,92309,[유캐슬] 카레나 엔틱 식탁세트_6인,식탁+주방가구,식탁,6인이상,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
2,92310,[유캐슬] 카레나 엔틱 식탁세트_4인,식탁+주방가구,식탁,4인,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0...
3,92311,[유캐슬] 루벤스 엔틱 식탁세트_6인,식탁+주방가구,식탁,6인이상,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...
4,92312,[유캐슬] 루벤스 엔틱 식탁세트_4인,식탁+주방가구,식탁,4인,https://image.hanssem.com/hsimg//gds/550/92/92...,[0 1 0 0 0 0 0 0 0],[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0...,[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0...


In [9]:
df.to_csv(path+".csv", index=False, encoding="utf-8-sig")

In [10]:
dropped_indices = []
imgs = []
lv2_names = []
for idx in tqdm(df.index):
    try:
        res = request.urlopen(df["IMG_URL"][idx])
        img = Image.open(BytesIO(res.read())) 
        if np.array(img).shape == (550,550,3) :
            imgs.append(img)
            lv2_name = df["LV2"][idx]
            lv2_names.append(lv2_name)
        else:
            dropped_indices.append(idx)
        
    except:
        dropped_indices.append(idx)

100%|██████████| 3102/3102 [09:37<00:00,  5.37it/s]


In [14]:
import pickle
dPath = "D:/download/csv/식탁-주방가구/식탁의자"

In [15]:
with open(dPath+"/imgs.p", "wb") as f:
    pickle.dump(imgs, f)

with open(dPath+"/lv2_names.p", "wb") as f:
    pickle.dump(lv2_names, f)

with open(dPath+"/dropped_indices.p", "wb") as f:
    pickle.dump(dropped_indices, f)

In [16]:

with open(dPath+"/dropped_indices.p", "rb") as f:
    dropped_indices = pickle.load(f)
len(dropped_indices)

13

In [17]:
with open(dPath+"/imgs.p", "rb") as f:
    imgs = pickle.load(f)
len(imgs)

3089

In [18]:
with open(dPath+"/lv2_names.p", "rb") as f:
    lv2_names = pickle.load(f)
len(lv2_names)

3089

In [19]:

df2 = df.copy()
if len(df2["IMG_URL"]) != len(imgs):
    df2 = df2.drop([df2.index[idx] for idx in dropped_indices])
    df2.to_csv(path+".csv", index=False, encoding="utf-8-sig")

In [20]:
len(df2["IMG_URL"]), len(imgs)

(3089, 3089)

In [21]:
for label in df["LV2"].unique():
    if not os.path.isdir(path+f"/{label}"):
        os.makedirs(path+f"/{label}")

In [22]:
for idx,img in tqdm(enumerate(imgs)):
    save_path = path+f"/{lv2_names[idx]}"
    # print(save_path)
    img.save(save_path+f"/{df['PROD_ID'][idx]}.png")

3089it [03:43, 13.79it/s]
